# Tutorial: Julia Basics

## Advanced Macroeconomics: Numerical Methods (MIE37)



### What is Julia

- developped at MIT on top of __opensource__ technologies
    - linux / git / llvm
- syntax inspired by Matlab but:
    - more consistent
    - lots of features from high level languages
- everything is JIT-compiled
    - interpreted vs compiled treadeoff
    - -> very fast
    - most of the base library is written in Julia
- opensource/free + vibrant community


Some useful links from QuantEcon:

* [Julia cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
* [Julia-Matlab comparison](https://cheatsheets.quantecon.org/index.html)
* [Julia essentials](https://lectures.quantecon.org/jl/julia_essentials.html)
* [Vectors, arrays and matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

Excellent resources at: [julialang](https://julialang.org/learning/)
- checkout JuliaAcademy, it's free
- ongoing [MOOC](https://computationalthinking.mit.edu/Spring21/) at MIT

### an example of what you shouldn't do in Matlab

How I learnt: interpreted code is slow, so vectorize your coe.

In [1]:
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end
@time [ stupid_loop(1000,1000,i) for i =1:10]

  0.108801 seconds (66.67 k allocations: 3.520 MiB, 24.75% compilation time)


10-element Vector{Float64}:
 1.0e6
 2.0e6
 3.0e6
 4.0e6
 5.0e6
 6.0e6
 7.0e6
 8.0e6
 9.0e6
 1.0e7

Code is translated to LLVM code then to instructions for the processor. Note that processor instructions are shorter than LLVM code.

In [2]:
@code_llvm stupid_loop(10,10,10)

;  @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:1 within `stupid_loop`
define double @julia_stupid_loop_1277(i64 signext %0

, i64 signext %1, i64 signext %2) #0 {
top:
;  @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:3 within `stupid_loop`
; ┌ @ range.jl:5 within `Colon`
; │┌ @ range.jl:393 within `UnitRange`
; ││┌ @ range.jl:400 within `unitrange_last`
     %.inv = icmp sgt i64 %0, 0
     %. = select i1 %.inv, i64 %0, i64 0
; └└└
  br i1 %.inv, label %L17.preheader, label %L94

L17.preheader:                                    ; preds = %top
  %.inv26 = icmp sgt i64 %1, 0
  %.24 = select i1 %.inv26, i64 %1, i64 0
  %.inv27 = icmp sgt i64 %2, 0
  %.25 = select i1 %.inv27, i64 %2, i64 0
;  @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:4 within `stupid_loop`
  %3 = select i1 %.inv26, i1 %.inv27, i1 false
  br i1 %3, label %L35.preheader.split.us.us.us, label %L94

L81.us.us:                                        ; preds = %L68.us.us.us
;  @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:9 within `stupid_loop`
; ┌ @ range.jl:883 within `iterate`
; │┌ @ promotion.jl:477 within `==`
    %.not29.us.u

In [3]:
@code_native stupid_loop(10,10,10)

	.text
	.file	"stupid_loop"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_stupid_loop_1312
.LCPI0_0:
	.quad	0x3ff0000000000000              # double 1
	.text
	.globl	julia_stupid_loop_1312
	.p2align	4, 0x90
	.type	julia_stupid_loop_1312,@function
julia_stupid_loop_1312:                 # @julia_stupid_loop_1312
; ┌ @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:1 within `stupid_loop`
	.cfi_startproc
# %bb.0:                                # %top
	vxorpd	%xmm0, %xmm0, %xmm0
; │ @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:3 within `stupid_loop`
; │┌ @ range.jl:5 within `Colon`
; ││┌ @ range.jl:393 within `UnitRange`
; │││┌ @ range.jl:400 within `unitrange_last`
	testq	%rdi, %rdi
; │└└└
	jle	.LBB0_9
# %bb.1:                                # %L17.preheader
	testq	%rsi, %rsi
; │ @ /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:4 within `stupid_loop`
	jle	.LBB0_9
# %bb.2:                                # %L17.preheade

### Syntax Review

#### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [4]:
# Assign the value 10 to the variable x
x = 10

10

In [5]:
x

10

In [8]:
2 == 3

false

In [10]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
σ = 34
🦆 = 23
🦈 = σ + 🦆
ϵ = 1e-4

0.0001

Default semantic is pass-by-reference:

In [ ]:
a = [1,2,3,4]
b = a
a[1] = 10
b

To work on a copy: `copy` or `deepcopy`

In [11]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

4-element Vector{Int64}:
 1
 2
 3
 4

In [12]:
a .== b

4-element BitVector:
 0
 1
 1
 1

In [13]:
c = b

4-element Vector{Int64}:
 1
 2
 3
 4

In [18]:
b = [1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [16]:
a .== b

4-element BitVector:
 1
 1
 1
 1

In [19]:
c === b

false

#### Basic types

In [22]:
# for any object `typeof` returns the type
?any

UndefVarError: UndefVarError: help not defined
  Welcome to Julia 1.8.4. The full manual is available at

  https://docs.julialang.org

  as well as many great tutorials and learning resources:

  https://julialang.org/learning/

  For help on a specific function or macro, type ? followed by its name, e.g.
  ?cos, or ?@time, and press enter. Type ; to enter shell mode, ] to enter
  package mode.

  To exit the interactive session, type CTRL-D (press the control key together
  with the d key), or type exit().

In [23]:
typeof(a)

Vector{Int64} (alias for Array{Int64, 1})

#### Numbers

In [24]:
y = 2 + 2

4

In [25]:
-y

-4

In [26]:
0.34*23

7.82

In [29]:
3//4 + 2//3

17//12

In [31]:
# Scalar multiplication doesn't require *
3(4 - 2)

6

In [ ]:
x = 4
2x

In [ ]:
typeof(x)

In [ ]:
sizeof(a)

#### Booleans

Equality

In [32]:
0 == 1

false

In [33]:
2 != 3

true

In [34]:
3 <= 4

true

Identity

In [ ]:
a = [34, 35]
b = [34, 35]
c = a

In [ ]:
c === a

In [ ]:
b === a

Boolean operator

In [35]:
true && false

false

In [36]:
true || false

true

In [37]:
!true

false

#### Strings

In [38]:
# Strings are written using double quotes
str = "This is a string"

"This is a string"

In [40]:
ch = '🦆' # this is a character

'🦆': Unicode U+1F986 (category So: Symbol, other)

In [ ]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

In [41]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a)"

"2 + 2 = 4"

In [42]:
println("It took me $(a) iterations")

It took me 4 iterations


In [43]:
# String concatenation using *
"hello" * "world"

"helloworld"

In [44]:
println("hello ", "world")

hello world


#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [45]:
A = [1, 2]

2-element Vector{Int64}:
 1
 2

All elements have the type:

In [46]:
A = [1, 1.4]

2-element Vector{Float64}:
 1.0
 1.4

In [47]:
typeof(A) == Vector{Int64}

false

In [49]:
A''

2-element Vector{Float64}:
 1.0
 1.4

To get the size of an array:

In [50]:
size(A)

(2,)

Arrays are *mutable*

In [51]:
A[1] = 10

10

In [52]:
A

2-element Vector{Float64}:
 10.0
  1.4

Julia has one-based indexing: you refer to the first element as 1 ($\neq$ zero-based indexing in C or Python)

In [54]:
A[2]

1.4

Arrays are mutable and their size can be changed too:

In [58]:
push!(A, 29)
A

6-element Vector{Float64}:
 10.0
  1.4
 29.0
 29.0
 29.0
 29.0

In [27]:
A

3-element Array{Float64,1}:
 10.0
  1.4
 29.0

Two comments:
- the `push!` operation is *fast*
- `!` is a julia convention to express the fact that `push!` mutates its first argument

#### tuples

In [59]:
size(A)  # is a tuple

(6,)

In [60]:
# you can create tuples with (,,,)
t = (1,2,3,4)


(1, 2, 3, 4)

In [31]:
t

(1, 2, 3, 4)

tuples differ from arrays in two ways:
- they are immutable
- they can contain non-homogenous objects

In [63]:
t[1]

1

In [62]:
t[1] = 2

MethodError: MethodError: no method matching setindex!(::NTuple{4, Int64}, ::Int64, ::Int64)

In [65]:
typeof((1, "1", [1]))

Tuple{Int64, String, Vector{Int64}}

2d arrays are also called matrices... and can be used for matrix multiplications.

In [68]:
[3 4; 5 6]

2×2 Matrix{Int64}:
 3  4
 5  6

In [83]:
a1 = [1,2,3,4]
a2 = [1,2,3,4]  .+ 4
[a1 ;; a2]
# cat(a1, a2; dims=2)

4×2 Matrix{Int64}:
 1  5
 2  6
 3  7
 4  8

In [37]:
b = [1 0.6 0]

1×3 Array{Float64,2}:
 1.0  0.6  0.0

In [ ]:
B = [0.1 0.2 0.3; 4 5 6]

Other ways to construct arrays:

In [78]:
# zero array
t = zeros(2,3)
t[1,2] = 23.2
t

2×3 Matrix{Float64}:
 0.0  23.2  0.0
 0.0   0.0  0.0

In [79]:
# random array (uniform distribution)
t= rand(3,3)
t

3×3 Matrix{Float64}:
 0.151296  0.390327  0.239194
 0.726286  0.371063  0.133779
 0.037311  0.183624  0.72499

In [46]:
# random array (normal distribution)
t= randn(3,3)
t

3×3 Array{Float64,2}:
 -0.149832     0.973627  -0.407871
 -0.00251947  -1.46936   -0.141511
  0.676479    -0.774655   0.349923

Vectorized operations take a ., even comparisons (pointwise operations)

In [84]:
B = [1 2;3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [85]:
B*B

2×2 Matrix{Int64}:
  7  10
 15  22

In [86]:
B .* B

2×2 Matrix{Int64}:
 1   4
 9  16

In [87]:
f(x) = x^2+1

f (generic function with 1 method)

In [88]:
f(43)

1850

In [89]:
f.(B)

2×2 Matrix{Int64}:
  2   5
 10  17

Elements are always accessed with square brackets:

In [90]:
B = [1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [ ]:
You get element $B_{ij}$ with `B[i,j]`

In [55]:
B[1,2]

2

You select a whole row/column with `:`

In [91]:
B

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [92]:
B[:,1]

2-element Vector{Int64}:
 1
 4

In [93]:
B[1,:]

3-element Vector{Int64}:
 1
 2
 3

In [94]:
B[:,1:2]

2×2 Matrix{Int64}:
 1  2
 4  5

In [96]:
B[:,1:end-1]

2×2 Matrix{Int64}:
 1  2
 4  5

#### Control flow

Conditions

In [97]:
x = 0
if x<0
    # block
    println("x is negative")
elseif (x > 0) # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

x is zero


While

In [67]:
i = 3
while i > 0
    println(i)
    i -= 1 # decrement
end

3
2
1


For loops: your iterate over any iterable object:
- range   `i1:i2`
- vector
- tuple

In [98]:
# Iterate through ranges of numbers
for i ∈ (1:3)
    println(i)
end

1
2
3


In [99]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city in cities
    println(city)
end

Boston
New York
Philadelphia


In [100]:
cities

3-element Vector{String}:
 "Boston"
 "New York"
 "Philadelphia"

In [101]:
states = ["Massachussets", "New York", "Pennsylvania"]

3-element Vector{String}:
 "Massachussets"
 "New York"
 "Pennsylvania"

In [103]:
two_by_two_iterable = zip(cities, states)

zip(["Boston", "New York", "Philadelphia"], ["Massachussets", "New York", "Pennsylvania"])

In [104]:
collect(two_by_two_iterable)

3-element Vector{Tuple{String, String}}:
 ("Boston", "Massachussets")
 ("New York", "New York")
 ("Philadelphia", "Pennsylvania")

In [105]:
[two_by_two_iterable...]

3-element Vector{Tuple{String, String}}:
 ("Boston", "Massachussets")
 ("New York", "New York")
 ("Philadelphia", "Pennsylvania")

In [106]:
# Iterate through arrays of tuples using zip
for kw in zip(cities, states)
    println(kw)
end

("Boston", "Massachussets")
("New York", "New York")
("Philadelphia", "Pennsylvania")


In [107]:
# Iterate through arrays of tuples using zip
for (city, state) in zip(cities, states)
    println("$city, $state")
end

Boston, Massachussets
New York, New York
Philadelphia, Pennsylvania


In [108]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City $i is $city")
end

City 1 is Boston
City 2 is New York
City 3 is Philadelphia


In [110]:
t = (;a = 2, b = 3)
t[1]
t.a

2

#### List comprehensions

In [111]:
[1:10. ...] # unpack operator

10-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0

In [112]:
[i^2 for i  in 1:10] # collect with comprehension syntax

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [118]:
[i^2 for i=1:100000000 if mod(i,2)==0] ;

In [120]:
gen = (i^2 for i=1:100000000 if mod(i,2)==0)

Base.Generator{Base.Iterators.Filter{var"#42#44", UnitRange{Int64}}, var"#41#43"}(var"#41#43"(), Base.Iterators.Filter{var"#42#44", UnitRange{Int64}}(var"#42#44"(), 1:100000000))

In [121]:
sum(gen)

338960700901149440

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

In [1]:
# Type definition with 4 fields
struct ParameterFree
    value  
    transformation  
    tex_label
    description 
end

In [2]:
pf = ParameterFree("1", x->x^2, "\\sqrt{1+x^2}", ("a",1))

ParameterFree("1", var"#9#10"(), "\\sqrt{1+x^2}", ("a", 1))

In [3]:
pf.value

"1"

Two reasons to create structures:
- syntactic shortcut (you access the fields with .)
- specify the types of the fields

In [4]:
# Type definition
struct Parameter
    value ::Float64
    transformation ::Function # Function is a type!
    tex_label::String
    description::String
end

In [5]:
p = Parameter("1", x->x^2, "\\sqrt{1+x^2}", ("a",1))

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:273
  ...

In [6]:
p = Parameter(0.43, x->x^2, "\\sqrt{1+x^2}", "This is a description")

Parameter(0.43, var"#13#14"(), "\\sqrt{1+x^2}", "This is a description")

In [7]:
p.value

0.43

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [9]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\\beta", "Discount rate")

Parameter(0.9, identity, "\\beta", "Discount rate")

In [10]:
function Parameter(value)
    return Parameter(value, x->x, "x", "Anonymous")
end

Parameter

In [11]:
Parameter(0.4)

Parameter(0.4, var"#15#16"(), "x", "Anonymous")

In [12]:
Parameter(value, transformation, tex) = Parameter(value, transformation, tex, "no description")

Parameter

In [13]:
methods( Parameter )

# 4 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:3
[2] Parameter(value) in Main at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:1
[3] Parameter(value, transformation, tex) in Main at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:1
[4] Parameter(value, transformation, tex_label, description) in Main at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:3

In [19]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Parameter(0.5, identity, "\alpha", "No description available")

Now the function `Parameter` has two different `methods` with different signatures:

In [9]:
methods(Parameter)

# 4 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at In[1]:3
[2] Parameter(value::Float64, tex_label::String) in Main at In[8]:4
[3] Parameter(value, transformation, tex) in Main at In[5]:1
[4] Parameter(value, transformation, tex_label, description) in Main at In[1]:3

We have seen that a function can have several implementations, called methods, for different number of arguments, 
or for different types of arguments.

In [15]:
fun(x::Int64, y::Int64) = x^3 + y

fun (generic function with 1 method)

In [16]:
fun(x::Float64, y::Int64) = x/2 + y

fun (generic function with 2 methods)

In [17]:
fun(2, 2)

10

In [18]:
fun(2.0, 2)

3.0

In [20]:
α.tex_label

"\alpha"

In [21]:
# Access a particular field using .
α.value

0.5

In [22]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

ErrorException: setfield!: immutable struct of type Parameter cannot be changed

### Mutable vs non mutable types


by default structures in Julia are non-mutable

In [23]:
p.value = 3.0

ErrorException: setfield!: immutable struct of type Parameter cannot be changed

In [24]:
mutable struct Params
    x:: Float64
    y:: Float64
end

In [25]:
pos = Params(0.4, 0.2)

Params(0.4, 0.2)

In [28]:
pos.x = "pos"

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:273
  ...

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [29]:
typeof( [1,2,3])

Vector{Int64} (alias for Array{Int64, 1})

In [30]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

In [31]:
Duple(3, 3)

Duple{Int64}(3, 3)

In [32]:
Duple(1//2, 2//3)

Duple{Rational{Int64}}(1//2, 2//3)

In [33]:
Duple(3, -1.0)

MethodError: MethodError: no method matching Duple(::Int64, ::Float64)
Closest candidates are:
  Duple(::T, !Matched::T) where T at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:3

In [34]:
struct Truple{T}
    x::Duple{T}
    z::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [35]:
sizeof(3.0)

8

In [36]:
sizeof( Truple(Duple(3.0, -15.0), 3.0) )

24

In [ ]:
# What happens here?
Duple(1.5, 3)

In [35]:
struct Truple3{T,S}
    x::Tuple{T,S}
    z::S
end

We can also restrict the type parameter `T`:

In [37]:
typeof("S") <: Number

false

In [38]:
typeof(4) <: Number

true

In [39]:
# T can be any subtype of Number, but nothing else
struct PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [41]:
PlanarCoordinate("4th Ave", "14th St")

MethodError: MethodError: no method matching PlanarCoordinate(::String, ::String)

In [40]:
PlanarCoordinate(2//3, 8//9)

PlanarCoordinate{Rational{Int64}}(2//3, 8//9)

Arrays are an exemple of mutable, parameterized types

In [43]:
x = convert(Int8, 4)

4

In [60]:
factorial(BigInt(123))

12146304367025329675766243241881295855454217088483382315328918161829235892362167668831156960612640202170735835221294047782591091570411651472186029519906261646730733907419814952960000000000000000000000000000

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [61]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

-25554.110202663698

In [64]:
@time sumofsins_unstable(Int(1e5))

  0.000237 seconds


-25554.110202663698

In [65]:
@time sumofsins_stable(Int(1e5))

  0.000119 seconds


-25554.110202663698

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

In [66]:
@code_warntype sumofsins_stable(Int(1e5))

MethodInstance for sumofsins_stable

(::Int64)
  from sumofsins_stable(n::Integer) in Main at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:10
Arguments
  #self#::Core.Const(sumofsins_stable)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  sum::Float64
  i::Int64
Body::Float64


1 ─       

(sum = 0.0)


│   %2  = (1:n)

::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = sum::Float64
│   %11 = Main.sin(3.4)::Core.Const(-0.2555411020268312)
│         (sum = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return sum



In [67]:
@code_warntype sumofsins_unstable(Int(1e5))

MethodInstance for sumofsins_unstable(::Int64)
  from sumofsins_unstable(n::Integer) in Main at /workspaces/mie37/notebooks/1_Julia_Basics.ipynb:2
Arguments
  #self#::Core.Const(sumofsins_unstable)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  sum::Union{Float64, Int64}
  i::Int64
Body::Union{Float64, Int64}
1 ─       (sum = 0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = sum::Union{Float64, Int64}
│   %11 = Main.sin(3.4)::Core.Const(-0.2555411020268312)
│         (sum = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return sum



### Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [68]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

print_type (generic function with 1 method)

In [69]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

print_type (generic function with 2 methods)

In [54]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

print_type (generic function with 3 methods)

In [55]:
# See all methods for a given function
methods(print_type)

# 3 methods for generic function "print_type":
[1] print_type(x::String) in Main at In[53]:3
[2] print_type(x::Number) in Main at In[51]:3
[3] print_type(x::Number, y::Number) in Main at In[54]:3

Julia uses **multiple dispatch** to decide which **method** of a **function** to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [56]:
print_type(5)

5 is a number


In [57]:
print_type("foo")

foo is a string


In [58]:
print_type([1, 2, 3])

MethodError: MethodError: no method matching print_type(::Array{Int64,1})
Closest candidates are:
  print_type(!Matched::String) at In[53]:3
  print_type(!Matched::Number) at In[51]:3
  print_type(!Matched::Number, !Matched::Number) at In[54]:3

#### Other types of functions

Julia supports a short function definition for one-liners

In [70]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

f (generic function with 2 methods)

As well as a special syntax for anonymous functions

In [71]:
u -> u^2

#17 (generic function with 1 method)

In [72]:
map(u->u^2, [1,2,3,4])

4-element Vector{Int64}:
  1
  4
  9
 16

In [77]:
function fun(arg1 ; add=0.0)
    x = arg1 + add
    return x +add
end

fun (generic function with 3 methods)

### Keyword arguments and optional arguments

In [79]:
fun(1; add=12)

25

### Packing/unpacking


In [80]:
t = (1,2,4)

(1, 2, 4)

In [81]:
a,b,c = t

(1, 2, 4)

In [82]:
[(1:10)...]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [66]:
cat([4,3], [0,1]; dims=1)

4-element Array{Int64,1}:
 4
 3
 0
 1

In [67]:
l = [[4,3], [0,1], [0, 0], [1, 1]]
# how do I concatenate it ?

cat(l...; dims=1) ### see python's f(*s)

8-element Array{Int64,1}:
 4
 3
 0
 1
 0
 0
 1
 1

### Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. _A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently._

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

## Additional Exercises

Taken from QuantEcon's [Julia Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1. Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using `enumerate`, write a function `p` such that `p(x, coeff)` computes the value of the polynomial with coefficients `coeff` evaluated at `x`.

ppp (generic function with 1 method)

2. Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations.